# Тестовые данные

In [2]:
# (input, output)
test_1 = ([[3,2,1],[1,7,6],[2,7,7]], 1)
test_2 = ([[3,1,2,2],[1,4,4,5],[2,4,2,2],[2,4,2,2]], 3)
test_3 = ([[3,2,1,1],[1,7,6,6],[2,7,7,7],[2,7,7,7]], 2)
test_4 = ([[1, 1], [1, 1]], 4)

# Кубический вариант

Cначала я решила додумать решение по подсказке, но куб остался :(

In [3]:
def count_equal_rowcol_pairs(grid):
    '''Сложность: O(n^3)'''
    
    n = len(grid)
    pairs_counter = 0
    
    # проходимся по строкам
    for row in grid:
        # для текущей строки проходимся по всем колонкам
        for j in range(n):
            for i in range(n):
                # как только находим неравные элементы - бросаем сравниваемую колонку
                if row[i] != grid[i][j]:
                    break
                # если проитерировались по всей колонке - значит найдена пара, т.к. все соответствующие элементы равны
                if i == n-1:
                    pairs_counter += 1
    
    return pairs_counter

In [9]:
assert count_equal_rowcol_pairs(test_1[0]) == test_1[1]
assert count_equal_rowcol_pairs(test_2[0]) == test_2[1]
assert count_equal_rowcol_pairs(test_3[0]) == test_3[1]
assert count_equal_rowcol_pairs(test_4[0]) == test_4[1]

Потом у меня было два промежуточных решения, мне казалось, что я нашла O(n^2), но когда доходило дело до реализации - я снова упиралась в какой-нибудь подкапотный O(n) и куб возвращался.

Я долго думала, как мне избавиться от куба и прийти к квадрату. Либо мне нужно как-то сравнивать списки, не перебирая n элементов, либо оставить сравнение списков за O(n), но оно должно быть внутри одного цикла, а не вложенного. 

# Квадратичный вариант

Мой мозг зацепился за сравнение списков за O(n) и вспомнил префиксное дерево (trie), в котором поиск слова происходит за O(n). Я прикинула, а что если я добавлю строки в trie, а потом буду искать колонки в этом trie? И, кажется, получился заветный квдрат :)

P.S.: кстати, с trie я столкнулась впервые, работая над дипломом в бакалавриате - на этой структуре сделан mystem от Яндекса :)

In [12]:
def count_equal_rowcol_pairs(grid):
    '''Сложность: O(n^2)'''
    
    n = len(grid)
    
    # добавление и поиск в trie: O(n)
    # trie[0] - переходы к другим вершинам
    # trie[1] - сколько раз дошли до этой вершины
    #   для всего, что не лист - 1,
    #   а для листа - этот счетчик плюсуется, т.к. нам нужно считать попадания в лист при построении дерева (кол-во одинаковых строк)
    trie = [dict(), 1]

    # строим trie из строк: O(n^2)
    for row in grid:
        current_node = trie
        for i, item in enumerate(row):
            try:
                current_node = current_node[0][item]
                # дошли до конца слова, которое уже было добавлено в дерево, поэтому плюсуем счетчик кол-ва таких слов
                if i == n-1:
                    current_node[1] += 1
            except KeyError:
                current_node[0][item] = [dict(), 1]
                current_node = current_node[0][item]

    # генерим колонки: O(n^2)
    columns = []
    for i in range(n):
        column = []
        for j in range(n):
            column.append(grid[j][i])
        columns.append(column)
    
    # обходм колонки и ищем их в trie из строк: O(n^2)
    pairs_counter = 0
    for column in columns:
        current_node = trie
        for i, item in enumerate(column):
            try:
                current_node = current_node[0][item]
                if i == n-1:
                    pairs_counter += current_node[1]
            except KeyError:
                break

    return pairs_counter

In [13]:
assert count_equal_rowcol_pairs(test_1[0]) == test_1[1]
assert count_equal_rowcol_pairs(test_2[0]) == test_2[1]
assert count_equal_rowcol_pairs(test_3[0]) == test_3[1]
assert count_equal_rowcol_pairs(test_4[0]) == test_4[1]

# В поисках O(n*log(n))

## Почему так не получится?

Я подозреваю, что быстрее чем за квадрат решить задачу не получится. Вот мои доводы:

**1. Предельный случай**

Допустим, у нас есть предельный случай, когда каждая строка равна каждой колонке. Например, grid = [[1, 1], [1, 1]]. Тогда, кол-во пар = n^2. Наш алгоритм должен найти все пары, значит он так или иначе сделает перебор в количестве n^2.

**2. Колонки**

Также я подозреваю, что сформировать колонки быстрее чем за квадрат не получится. Ведь матрица задана строками, а чтобы получить колонку, нужно из каждой строки (т.е. n раз) взять i-ый элемент (где 0 <= i < n). Таким образом получаем n^2, который перекроет любую оставшуюся часть алгоритма, даже если она будет быстрее.

Я попробовала придумать арифметическую штуку, чтобы вычислять нужный индекс в рамках одного цикла, но цикл-то все равно сделает n^2 итераций, так что она бесмысленна :)

In [4]:
# бессмысленная арифметическая штука
for i in range(n*n):
    k = i - (i // n)*n
    l = i // n

## А может все-таки получится?

Ниже описаны те идеи, которые приходили мне на ум при попытке свести решение к O(n*log(n)), и опровержение этих идей 🥲

Вообще, все описанное ниже не имеет особого смысла, т.к. эти решения предполагают, что я буду сравнивать колонки со строками, а заполучить колонки у меня получается за n^2 только :( 

### Сортировки

**со сложностью O(nlog(n))**

Когда я вижу nlog(n), то сразу думаю:
- о сортировках, которые отработают за O(nlog(n)) 
- и о том, как после сортировки буду искать n колонок бинарным поиском (log(n)), т.е. снова O(nlog(n))

Но! Сортировка списков в лексикографическом порядке даст дополнительную n, которая вылазит при сравнении списков, поэтому сортировки со сложностью O(nlog(n)) дадут в итоге сложность O((n^2)log(n)).

**radix sort**

Я вспомнила про radix sort, которая вроде как может отработать линейно. Попробовала адаптировать ее к кейсу с сортировкой списков:
- n - длина входного массива, т.е. кол-во строк в grid;
- d - длина строки в grid/кол-во колонок в grid, в нашем случае d=n;
- b - кол-во чисел, которые могут встретиться в grid. Теоретически их значения варьируются от 0 до 10^5, но на практике мы можем рассматривать только уникальные числа в grid, которых в худшем случае будет n^2 штук.

Посчитаем сложность: O(d(n+b)) = O(n(n+n^2)) = O(n^2 + n^3) = O(n^3) 🥲🥲🥲

Кстати, если вспомогательный массив заменить хэш-таблицей, то кажется можно получить такую сложность: O(dn) = O(n^2). Одним плачущим эмоджи меньше 🥲🥲

### Деревья

Близко к сортировкам лежит идея дерева поиска, типа бинарного или b tree и т.д. В таких деревьях я буду искать все колонки за O(nlog(n)), но опять-таки все сводиться к O((n^2)log(n)) из-за сравнения списков.

### Сравнение по хэшу

Сортировки и деревья упираются в сравнение списков за O(n). Тогда можно было бы сравнивать их хэши за O(1). Но высчитывание хэша от списка - O(n). И так нужно сделать для всех списокв n. Таким образом мы снова получим O(n^2) на этапе подготовки хэшей для сравнения списков :(

## Вывод

Если получится, то не у меня :D